In [1]:
import os
import yaml
import pandas as pd
from openai import ChatCompletion

In [2]:
# Função para carregar o schema.yml existente
def load_schema(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return yaml.safe_load(f)
    return {"version": 2, "models": []}

# Função para salvar o schema atualizado
def save_schema(file_path, schema):
    with open(file_path, "w") as f:
        yaml.dump(schema, f, sort_keys=False)

# Função para gerar descrições automáticas

def generate_description(column_name):
    prompt = f"Descreva de forma breve e objetiva o significado da coluna '{column_name}' em uma tabela de banco de dados."
    response = ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Você é um assistente que gera descrições curtas para colunas de tabelas."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"].strip()

# Função principal para atualizar schema.yml
def update_schema_yml(schema_path, folder_path):
    schema = load_schema(schema_path)
    existing_models = {model["name"]: model for model in schema["models"]}
    
    # Identificar arquivos .sql no diretório
    sql_files = [f for f in os.listdir(folder_path) if f.endswith(".sql")]
    new_models = []

    for sql_file in sql_files:
        model_name = sql_file.replace(".sql", "")
        model_path = os.path.join(folder_path, sql_file)
        
        # Ler colunas a partir da tabela
        with open(model_path, "r") as f:
            lines = f.readlines()
        columns = [line.split()[0].replace(",", "") for line in lines if " as " in line.lower()]

        # Adicionar modelo ao schema
        if model_name not in existing_models:
            new_model = {
                "name": model_name,
                "description": f"Descrição automática gerada para a tabela {model_name}.",
                "columns": [
                    {
                        "name": column,
                        "description": generate_description(column)
                    }
                    for column in columns
                ]
            }
            new_models.append(new_model)

    # Atualizar schema existente
    schema["models"] = new_models
    save_schema(schema_path, schema)


In [4]:
# Paths for your schema.yml files and SQL folders
core_schema_path = "C:/Users/larissalorenzi/Documents/GitHub/northwind_desafio/models/core/schema.yml"
stage_schema_path = "C:/Users/larissalorenzi/Documents/GitHub/northwind_desafio/models/stage/schema.yml"
core_sql_folder = "C:/Users/larissalorenzi/Documents/GitHub/northwind_desafio/models/core"
stage_sql_folder = "C:/Users/larissalorenzi/Documents/GitHub/northwind_desafio/models/stage"

In [5]:
# Update the schemas
update_schema(core_schema_path, core_sql_folder)
update_schema(stage_schema_path, stage_sql_folder)
print("Schemas updated successfully.")

Schemas updated successfully.


In [ ]:
# Exemplo de uso
schema_path = "schema.yml"  # Caminho para o arquivo schema.yml
models_path = "./models"  # Caminho para a pasta com arquivos .sql
update_schema_yml(schema_path, models_path)